In [1]:
!pip install --upgrade --user scipy


     ---------------------------------------- 0.0/60.8 kB ? eta -:--:--
     ------ --------------------------------- 10.2/60.8 kB ? eta -:--:--
     ------ --------------------------------- 10.2/60.8 kB ? eta -:--:--
     ------------- ------------------------- 20.5/60.8 kB 93.9 kB/s eta 0:00:01
     ------------- ------------------------- 20.5/60.8 kB 93.9 kB/s eta 0:00:01
     ------------------- ------------------ 30.7/60.8 kB 100.9 kB/s eta 0:00:01
     ------------------------- ------------ 41.0/60.8 kB 122.9 kB/s eta 0:00:01
     ------------------------- ------------ 41.0/60.8 kB 122.9 kB/s eta 0:00:01
     ------------------------- ------------ 41.0/60.8 kB 122.9 kB/s eta 0:00:01
     ------------------------- ------------ 41.0/60.8 kB 122.9 kB/s eta 0:00:01
     ------------------------- ------------ 41.0/60.8 kB 122.9 kB/s eta 0:00:01
     -------------------------------- ------ 51.2/60.8 kB 87.3 kB/s eta 0:00:01
     --------------------------------------- 60.8/60.8 kB 97.9

In [8]:
import numpy as np
from scipy.optimize import linprog

# Define cost per serving (Objective Function Coefficients)
cost = [2.5, 0.8, 1.5]  # Chicken, Rice, Broccoli

# Constraint Coefficients (Calories, Protein, Fat)
A = [
    [-250, -130, -50],  # Calories constraint
    [-30, -3, -4],  # Protein constraint
    [-8, -0.5, -0.2],  # Fat constraint
]

# Constraint Boundaries (Minimum Requirements)
b = [-2000, -50, -10]  # Negative values because linprog assumes Ax <= b

# Bounds (Non-negativity Constraint)
x_bounds = (0, None)  # Chicken
y_bounds = (0, None)  # Rice
z_bounds = (0, None)  # Broccoli

# Solve Linear Program
result = linprog(cost, A_ub=A, b_ub=b, bounds=[x_bounds, y_bounds, z_bounds], method='highs')

# Display Results
if result.success:
    print("Optimal Diet Plan:")
    print(f"Chicken Servings: {result.x[0]:.2f}")
    print(f"Rice Servings: {result.x[1]:.2f}")
    print(f"Broccoli Servings: {result.x[2]:.2f}")
    print(f"Total Cost: ${result.fun:.2f}")
else:
    print("Optimization failed.")

# Extension Task: Add Vitamin C Constraint
A.append([-0, -0, -50])  # Vitamin C contribution only from Broccoli
b.append(-30)  # Minimum Vitamin C requirement

# Solve Again with Vitamin C Constraint
result_vitaminC = linprog(cost, A_ub=A, b_ub=b, bounds=[x_bounds, y_bounds, z_bounds], method='highs')

# Display Updated Results
if result_vitaminC.success:
    print("\nOptimal Diet Plan with Vitamin C Constraint:")
    print(f"Chicken Servings: {result_vitaminC.x[0]:.2f}")
    print(f"Rice Servings: {result_vitaminC.x[1]:.2f}")
    print(f"Broccoli Servings: {result_vitaminC.x[2]:.2f}")
    print(f"Total Cost: ${result_vitaminC.fun:.2f}")
else:
    print("Optimization with Vitamin C constraint failed.")


Optimal Diet Plan:
Chicken Servings: 0.33
Rice Servings: 14.75
Broccoli Servings: 0.00
Total Cost: $12.62

Optimal Diet Plan with Vitamin C Constraint:
Chicken Servings: 0.33
Rice Servings: 14.52
Broccoli Servings: 0.60
Total Cost: $13.34


In [9]:
import numpy as np
from scipy.optimize import linprog

# Define cost per serving (Chicken, Rice, Broccoli)
cost = [2.50, 0.80, 1.50]  # Minimize: 2.5x + 0.8y + 1.5z

# Define nutrition constraints matrix (Calories, Protein, Fat, Vitamin C)
# Each row corresponds to a nutrient requirement
# Each column represents a food item (Chicken, Rice, Broccoli)
nutrition = [
    [250, 130, 50],   # Calories per serving
    [30, 3, 4],       # Protein per serving
    [8, 0.5, 0.2],    # Fat per serving
    [0, 0, 50]        # Vitamin C per serving (only from Broccoli)
]

# Define minimum daily intake for each nutrient
min_nutrition = [2000, 50, 10, 30]  # [Calories, Protein, Fat, Vitamin C]

# Convert constraints to standard form (Ax ≤ b)
A_ub = -1 * np.array(nutrition)  # Flip sign to convert ≥ to ≤
b_ub = -1 * np.array(min_nutrition)

# Define bounds for each food item (no negative servings)
bounds = [(0, None), (0, None), (0, None)]

# Solve the linear programming problem
result = linprog(c=cost, A_ub=A_ub, b_ub=b_ub, bounds=bounds, method="highs")

# Print results
if result.success:
    servings = result.x
    print("Optimal daily servings:")
    print(f"  Chicken: {servings[0]:.2f} servings")
    print(f"  Rice: {servings[1]:.2f} servings")
    print(f"  Broccoli: {servings[2]:.2f} servings")
    print(f"Total cost: ${result.fun:.2f}")
else:
    print("Optimization failed:", result.message)


Optimal daily servings:
  Chicken: 0.33 servings
  Rice: 14.52 servings
  Broccoli: 0.60 servings
Total cost: $13.34


In [ ]:
Before Adding Vitamin C Constraint:       
The optimization algorithm chose to completely avoid broccoli because it's not necessary to meet the calorie, protein, and fat requirements at the lowest cost.
The majority of the diet comes from rice (14.75 servings) because it's the cheapest calorie source.

In [ ]:
After Adding the Vitamin C Constraint:
Now that Vitamin C is required (at least 30 mg per day), the optimizer had to include some broccoli (0.60 servings) since it’s the only source of Vitamin C in the diet.
The amount of rice slightly decreased (from 14.75 to 14.52 servings) to adjust for the additional broccoli.
The total cost increased from $12.62 to $13.34 because broccoli was added.